In [3]:
import pytesseract
import cv2
from PIL import Image 
import re
import pandas as pd

In [4]:
pytesseract.pytesseract.tesseract_cmd = r"C:\Program Files\Tesseract-OCR\tesseract.exe"

In [196]:
formats = pd.read_excel('invoice_formats.xlsx')
final = pd.read_excel('Extracted_Data.xlsx')

In [238]:
Ima = "Makro Invoices/Makro Invoices/New/09.11.20_APPLUS ITEUVE TECHHNOLOGY S.L._60,1_page1.png"

In [231]:
%%time
dat = {
    "Vat Number":'',
       "Total":0,
       "Date":'',
      "Invoice_Number":''}
img  = cv2.imread(Ima, cv2.COLOR_RGB2GRAY)
text=(pytesseract.image_to_string(img))
text_list = text.split("\n")
vat = []
date = []
print(type(text_list))
for idx, a in enumerate(text_list):
    if(a.strip() != ""):


        if (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            print(b)
        elif (re.search("[0-9]{8}-[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}-[A-Z]{1}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
        elif (re.search("[0-9]{8}[A-Z]{1}",a)):
            b = re.findall("[0-9]{8}[A-Z]{1}",a)
            dat["Vat Number"] = b
            vat.append(b)
    
        elif (re.search("[A-Z]{1}-[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)

        elif (re.search("[A-Z]{1}[0-9]{8}",a)):
            b = re.findall("[A-Z]{1}[0-9]{8}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]
            vat.append(b)
            
        elif (re.search("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}.[0-9]{3}.[0-9]{3}",a)
            dat["Vat Number"] = b[0]
        elif (re.search("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)):
            b = re.findall("[A-Z]{1}-[0-9]{2}/[0-9]{6}",a)
            dat["Vat Number"] = b[0]

        if len(vat)>1:
            dat["Vat Number"] = vat[0]
        else:
            pass
         #Date section           
        if (re.search("\d{2}/\d{2}/\d{4}",a)):
            b  = re.findall("\d{2}/\d{2}/\d{4}", a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}-\d{2}-\d{4}",a)):
            b = re.findall("\d{2}-\d{2}-\d{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("\d{2}/\d{2}/\d{2}",a)):
            b = re.findall("\d{2}/\d{2}/\d{2}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        elif (re.search("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)):
            b = re.findall("[0-9]{2}.[0-9]{2}.[0-9]{4}",a)
            dat["Date"] = b[0]
            date.append(b[0])
        if len(date)>1:
            dat["Date"] = date[0]

        #invoice number section
        if ("Factura" in a or "ALBARAN" in a or "Factura: " in a):

            print(a)
            b = a.split(" ")
            if b[0] == "ALBARAN":
                dat["Invoice_Number"] = b[1]
            print(b)
            if len(b)>3:
                print(b)
                if len(b)>4 and b[2] !='—' and len(b[1])!=25 and len(b[1])>5:
                    dat["Invoice_Number"] = b[1]+b[2]
                    print(b)
                elif len(b)>=5 and len(b[1])==25:
                    dat["Invoice_Number"] = b[1]
                elif len(b)>6 and len(b[3])>15:
                    dat["Invoice_Number"] = b[1]+ b[2]+ b[3]
                elif len(b)>6 and b[1]== "0/0":
                    dat["Invoice_Number"] = b[1] + b[2]
                    
        if ("Tarjeta" in a or "Credito" in a or  "pagina" in a or "Total:" in a  or "TOTAL" in a or  "Total factura" in a or  "Total Factura" in a or "TOTAL FACTURA" in a or "Total CON IGIC" in a  or "IMPORTE FACTURA EU" in a or "IMPORTE FACTURA" in a or "EUROS" in a):
            print("Case Total")
            b = a.split(" ")
            print(b)
            if len(b[-1])==2:
                
                dat["Total"] = b[-2]+b[-1]
                print(b[-2]+b[-1])
                
            if b[-1]=='€':
                dat["Total"] = b[-2]
            if b[-1] == "|":
                dat["Total"] = b[-2]
            if b[-1] == "EUR":
                dat["Total"] = b[-2]
            else:
                dat["Total"] = b[-1]
                print(b[-1])
                
if dat["Total"] == 0 or len(dat['Total'])==1 or dat["Total"]=="SUB TOTAL"  or dat["Total"] == 'factura' or dat["Total"]=="FACTURA" or dat["Total"]== "EUROS":
        tot = []
        im = Image.open(Ima) 
        # Size of the image in pixels (size of orginal image) 
        # (This is not mandatory) 
        width, height = im.size 
        print(im.size)
        # Setting the points for cropped image 
        left = 605
        top = 600
        right = 1675
        bottom = 1590
        # Cropped image of above dimension 
        # (It will not change orginal image) 
        im1 = im.crop((left, top, right, bottom)) 
         # Shows the image in image viewer 
        im1.save("Makro Invoices/Makro Invoices/New/pp2.jpg")
        img  = cv2.imread('Makro Invoices/Makro Invoices/New/pp2.jpg', cv2.COLOR_RGB2GRAY)
        tex=(pytesseract.image_to_string(img,config='outputbase digits'))
        text_list = tex.split("\n")
        text_list = text_list
        print(text_list)
        for i in text_list:
            if len(i)>=4 and len(i)<=7:
                print(i)
                tot.append(i)
                dat["Total"] = max(tot)
            
            
        

<class 'list'>
Factura N° 9335
['Factura', 'N°', '9335']
Case Total
['SUBTOTAL', 'EUROS']
EUROS
Case Total
['TOTAL', 'EUROS']
EUROS
(1191, 1684)
['1.20000', '1.50000', '1.50000', '1.20000', '1.50000', '1.50000', '1.20000', '1.20000', '1.200.00', '1.20000', '1.500.00', '1.500.00', '', '50000', '', '50000', '180.00', '', '50.00', '', '20000', '', '50.00', '', '1.200.00', '1.500.00', '1.500.00', '1.200.00', '1.500.00', '1.500.00', '1.200.00', '1.200.00', '1.200.00', '1.200.00', '1.500.00', '1.500.00', '', '50000', '50000', '', '180.00', '', '3.47600', '', '25000', '', '20000', '', '150.00', '', '21.456.00', '', '1.501.92', '22.957.92', '', '']
1.20000
1.50000
1.50000
1.20000
1.50000
1.50000
1.20000
1.20000
1.20000
50000
50000
180.00
50.00
20000
50.00
50000
50000
180.00
3.47600
25000
20000
150.00
Wall time: 5.02 s


In [232]:
dat

{'Vat Number': 'B76546787',
 'Total': '50000',
 'Date': '22 78 0001',
 'Invoice_Number': ''}

In [233]:
data  =  pd.DataFrame(dat,index=[0])
data['Vat Number']= data['Vat Number'].str.replace(' ', '')
print(data)

  Vat Number  Total        Date Invoice_Number
0  B76546787  50000  22 78 0001               


In [234]:
data['Vat Number']= data['Vat Number'].str.replace('-', '')
data['Vat Number']= data['Vat Number'].str.replace(',', '')
data['Vat Number']= data['Vat Number'].str.replace('.', '')
data

F:\anaconda3\envs\python_3_7\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  This is separate from the ipykernel package so we can avoid doing imports until


,Vat Number,Total,Date,Invoice_Number
0,B76546787,50000,22 78 0001,


In [235]:
formats['last']= (formats['nif'].isin(data['Vat Number']))

In [236]:
df2 = formats.query("last == True")

In [237]:
df2

,formato,nombre,nif,last
411,[0-9]{12},La Creme Films SL,B76546787,True
5768,[0-9]{12},La Creme Films SL,B76546787,True
5774,[0-9]{12},La Creme Films SL,B76546787,True
7869,[0-9]{12},La Creme Films SL,B76546787,True
11762,[0-9]{12},La Creme Films SL,B76546787,True
21689,[0-9]{12},Graciosamar Cruceros,B76546787,True


In [213]:
inv = 0
for i in df2['formato']:
    inv = i
pat = 'F1-1-01353'
factura = re.compile(inv)
mo = factura.search(text)
dat["Invoice_Number"]=mo.group()

In [214]:
dat

{'Vat Number': 'A38003844',
 'Total': '104,14',
 'Date': ['08/07/20'],
 'Invoice_Number': 'FV20-29963'}

In [154]:
text

'VIVEROS MOGAN S.L\nCL. POSTREAGUA, S/N\n\n35140 MOGAN\nLAS PALMAS\n\nCIFINIF: B35129584\n\nVIVERO LA COSMA S.L\nCL. LAFUENTE, 18\n\n38260 TEJINA\n\nTel: 682272915, Fax:\n\nCIFINIF: B38960274\n\nN° fact ‘echa\n\nFavraa0s | 0st 1/2020 == Nuestras plantas son productos vivos. Enlregados en perfectas\n\nRenresentants condiciones.\n. - No nos hacemos cargo de las perdidas causadas por las condiciones a\nLA EMPRESA‘ que sean expuestas una vez entregadas y fuera del vivero.\n\n3 8 GUAYABITO LIMON (Psidium litorale) 9,80 3 78,40\n81 8 GUAYABO FRESA (Psidium cattleianum) 9,80 3 78,40\n14 5 CARAMBOLA (Averrhoa carambola) 15,00 3 75,00\n25 2 MAMEY COLORADO (Pouteria sapota) 24,80 3 49,60\n65 8 GUAYABO SP (PSIDIUM GUAJAVA) 9,80 3 78,40\n49 3 ZAPOTE BLANCO (Casimiroa edulis) 20,00 3 60,00\n150 410 LIMAM.17 10,50 3 105,00\n140 410 MANDARINO M.17 10,50 3 105,00\n121 60 PAPAYAM.14 (Carica papaya) 2,30 3 138,00\n250 412 PLATANERAM.14 4,60 3 55,20\n187 24 PINATROPICAL\n\n“Base imponible Ici ~Requiv. | 